In [1]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'
require 'cutorch'
require 'cunn'
require 'cunnx'

In [2]:
model = 'lstm'
nepochs = 200
K_tokens = 20
J_sentences = 10
batch_size = 100
thresh = 0.00
embed_dim = 100
learning_rate = 0.1
print_every = 1
epsilon = 1
usecuda = false
epsilon = 1
cuts = 4
base_explore_rate = 0.0
skip_rate = 0.
metric = 'f1'

In [3]:
--- Loading utility script
dofile("utils.lua")
dofile("model_utils.lua")

torch.manualSeed(420)
math.randomseed(420)

data_path = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/'

query_fn = data_path .. 'queries_numtext.csv'
query_file =  csvigo.load({path = query_fn, mode = "large", verbose = false})
queries = buildTermDocumentTable(query_file, nil)

pakistan = {
        ['inputs'] = '2012_pakistan_garment_factory_fires_first_sentence_numtext2.csv',
        ['nuggets'] ='pakistan_nuggets_numtext.csv',
        ['query'] = queries[2],
        ['query_name'] = 'pakistan'
}
aurora = {
        ['inputs'] = '2012_aurora_shooting_first_sentence_numtext2.csv', 
        ['nuggets'] = 'aurora_nuggets_numtext.csv',
        ['query'] = queries[3],
        ['query_name'] = 'aurora'
}
sandy = {
        ['inputs'] = 'hurricane_sandy_first_sentence_numtext2.csv',
        ['nuggets'] ='sandy_nuggets_numtext.csv',
        ['query'] = queries[7],
        ['query_name'] = 'sandy'
}

inputs = {
        aurora, 
        -- pakistan,
        -- sandy
    }
--- Only using epsilon greedy strategy for (nepochs/cuts)% of the epochs
delta = 1./(nepochs/cuts) 
crit = nn.MSECriterion()

...Utils file loaded	


In [4]:
input_path, query_file, batch_size, nepochs, inputs = data_path, query_file, batch_size, nepochs, inputs
model, crit, thresh, embed_dim, epsilon, delta = model, crit, thresh, embed_dim, epsilon, delta
base_explore_rate, print_every, learning_rate, J_sentences, K_tokens, use_cuda =base_explore_rate, print_every,learning_rate, J_sentences, K_tokens, usecuda
skiprate, emetric =  skip_rate, metric

In [5]:
if use_cuda then
  Tensor = torch.CudaTensor
  LongTensor = torch.CudaLongTensor
  crit = crit:cuda()
  print("...running on GPU")
else
  Tensor = torch.Tensor
  LongTensor = torch.LongTensor
  print("...running on CPU")
end

...running on CPU	


In [6]:
vocab_size = 0
maxseqlen = 0
maxseqlenq = getMaxseq(query_file)

action_query_list = {}
yrougue_query_list = {}
pred_query_list = {}

for query_id = 1, #inputs do
    input_fn = inputs[query_id]['inputs']
    nugget_fn = inputs[query_id]['nuggets']

    input_file = csvigo.load({path = input_path .. input_fn, mode = "large", verbose = false})
    nugget_file = csvigo.load({path = input_path .. nugget_fn, mode = "large", verbose = false})

    vocab_sized = getVocabSize(input_file)
    vocab_sizeq = getVocabSize(query_file)
    vocab_size = math.max(vocab_size, vocab_sized, vocab_sizeq)

    maxseqlend = getMaxseq(input_file)
    maxseqlen = math.max(maxseqlen, maxseqlenq, maxseqlend)
    action_list = torch.totable(torch.round(torch.rand(#input_file)))
    action_list[1] = 0

    --- initialize the query specific lists
    action_query_list[query_id] = action_list
    yrougue_query_list[query_id] = torch.totable(torch.randn(#input_file, 2)) --- Actual
    pred_query_list[query_id] = torch.totable(torch.zeros(#input_file, 2))    --- Predicted
end

In [7]:
-- model  = build_model(model, vocab_size, embed_dim, 2, use_cuda)
model  = build_model2(model, vocab_size, embed_dim, 1, use_cuda)

Running LSTM model	


In [ ]:
epoch = 0
minibatch = 1
query_id = 1

In [ ]:
unpackZeros(torch.totable(summary:t()[1])),
unpackZeros(torch.totable(sentences:t()[1])),
actions[1]

In [ ]:
tst2 = buildCurrentSummary(torch.totable(actions1), xout, K_tokens * J_sentences)

In [ ]:
tst2[1]

In [ ]:
function buildCurrentSummary(preds, xs, K)
    local out = {}
    for i = 1, #xs do
        if i == 1 then
            out[i] = {}
        elseif i == 2 then
            tmp = x_or_pass(preds[i-1], unpackZeros(xs[i-1])) 
            out[i] = getLastKElements(tmp, K)
        else 
            local tmp = tableConcat(out[i-2], x_or_pass(preds[i-1], unpackZeros(xs[i-1])))
            out[i] = getLastKElements(tmp, K)
        end 
    end
    return out 
end

function buildPredSummary2(preds, xs, K)
    --- This function is used to map the token indices to extract the summary
    --- and produceds {token_id, 0, token_id} from any given *selected* sentence
    local out = {}
    for i=1, #xs do
        if i == 1 then 
            out[i] = x_or_pass(preds[i][1], unpackZeros(xs[i]))
        else 
            --- Update it by adding xs_i and out_{i-1}
            local tmp = tableConcat(out[i-1], x_or_pass(preds[i][1], unpackZeros(xs[i])))
            --- Getting the last K tokens because we want to keep last K tokens
            out[i] =  getLastKElements(tmp, K)
        end
    end
    return out
end

In [ ]:
tst1 = buildPredSummary(torch.totable(actions1), xout, K_tokens * J_sentences)

In [ ]:
atmp = geti_n(torch.totable(actions1),4, 6 )
xtmp = geti_n(xout, 4, 6)

In [ ]:
atmp, xtmp

In [ ]:
atmp[1]

In [ ]:
x_or_pass(atmp[1], xtmp[1])

In [ ]:
actions1[1], buildPredSummary(geti_n(torch.totable(actions1),1, 3 ), 
    geti_n(xout, 1, 3), K_tokens * J_sentences)

In [ ]:
actions1[1], buildPredSummary(torch.totable(actions1)[1], {xout[1]}, K_tokens * J_sentences)

In [ ]:
actions0[1], buildPredSummary(torch.totable(actions0)[1], {xout[1]}, K_tokens * J_sentences)

In [ ]:
summary1[1], actions1[1]

In [ ]:
loss = 0.                    --- Compute a new MSE loss each time
--- Grabbing all of the input data
qs = inputs[query_id]['query']
input_file = csvigo.load({path = input_path .. inputs[query_id]['inputs'], mode = "large", verbose = false})
nugget_file = csvigo.load({path = input_path .. inputs[query_id]['nuggets'], mode = "large", verbose = false})
nuggets = buildTermDocumentTable(nugget_file, nil)

--- Building table of all of the input sentences
xtdm  = buildTermDocumentTable(input_file, K_tokens)

--- Extracting the query specific summaries, actions, and rougue
action_list = action_query_list[query_id]
yrouge = yrougue_query_list[query_id] 
preds = pred_query_list[query_id] 

nbatches = torch.floor( #input_file / batch_size)

--- Initializing rouge metrics at time {t-1} and save scores, reset each new epoch